# Import Library 

In [ ]:
import os
import json
import requests
import datetime
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor
from typing import Optional
from pydantic import BaseModel, Field

In [32]:
load_dotenv()

True

# Coding

## General Function to Get Data From Sector

In [33]:
GROQ_API_KEY = os.getenv("GROK_API_KEY")
SECTORS_API_KEY = os.getenv("SECTOR_API_KEY")

def retrieve_from_endpoint(url: str) -> dict:
    headers = {"Authorization": SECTORS_API_KEY}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)

## Tool Function

In [34]:
@tool

def get_company_overview(stock:str)-> str:
    """
    Get company overview from sector API
    """

    url = f"https://api.sectors.app/v1/company/report/{stock}/?sections=overview"
    
    return retrieve_from_endpoint(url)

In [35]:
@tool
def get_today_date()->str:
    """
    Get today's date
    """
    return datetime.datetime.now().date().strftime("%Y-%m-%d")

In [36]:
@tool
def get_top_companies_by_tx_volume(start_date: str, end_date: str, top_n: int = 5) -> str:
    """
    Get top companies by transaction volume
    """
    url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}"

    return retrieve_from_endpoint(url)

In [37]:
@tool
def get_daily_tx(stock: str, start_date: str, end_date: str) -> str:
    """
    Get daily transaction for a stock
    """
    url = f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}"

    return retrieve_from_endpoint(url)

In [38]:
tools = [
    get_company_overview,
    get_top_companies_by_tx_volume,
    get_daily_tx,
    get_today_date
]

In [39]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Answer the following queries, being as factual and analytical 
            as you can. If you need the start and end dates but they are not 
            explicitly provided, infer from the query. Whenever you return a 
            list of names, return also the corresponding values for each name. 
            If the volume was about a single day, the start and end 
            parameter should be the same. today data means today's date based on computer time"""
        ),
        ("human", "{input}"),
        # msg containing previous agent tool invocations 
        # and corresponding tool outputs
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [40]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

In [41]:
agent = create_tool_calling_agent(llm, tools, prompt)


In [42]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [43]:
query_1 = "What are the top 3 companies by transaction volume over the last 7 days from today?"
query_2 = "Based on the closing prices of BBCA between 1st and 30th of June 2024, are we seeing an uptrend or downtrend? Try to explain why."
query_3 = "What is the company with the largest market cap between BBCA and BREN? For said company, retrieve the email, phone number, listing date and website for further research."
query_4 = "What is the performance of GOTO (symbol: GOTO) since its IPO listing?"
query_5 = "If i had invested into GOTO vs BREN on their respective IPO listing date, which one would have given me a better return over a 90 day horizon?"
query_6 = "What today date?"

queries = [query_6, query_1, query_2, query_3, query_4, query_5]

# batch_input = [{"input": query} for query in queries]
# results = agent_executor.batch(batch_input)
# results

# for i in results:
#     print(i["output"])


# Structure Output

In [44]:
class Stock(BaseModel):
    """Information about a company's stock"""

    symbol: str = Field(description="The stock symbol")
    name: str = Field(description="The name of the company for which the stock symbol represents")
    sector: Optional[str] = Field(default=None, description="The sector of the company")
    industry: Optional[str] = Field(default=None, description="The industry of the company")
    market_cap: Optional[int] = Field(default=None, description="The market capitalization of the company")



In [45]:
agent_structured_output = agent.with_structured_output(Stock)

# Define the stock symbol
symbol = "BBCA"

# Inject the symbol variable into the text using an f-string
text = f"give me information for the stock: {symbol} in Indonesia stock market"

print(text)

AttributeError: 'RunnableSequence' object has no attribute 'with_structured_output'

In [ ]:
result = agent_executor.invoke({"input": text})

# Attempt to parse the result as a Stock model
try:
    stock_info = Stock(**result["output"])  # Unpack the output dictionary into the Stock model
    print("Structured Stock Output:")
    print(stock_info.json(indent=4))
except Exception as e:
    print("Failed to parse output as Stock model:", e)